In [103]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from random import randint
from time import time,sleep
from datetime import datetime, timedelta
import schedule

def scrape_forex_data():

    # This section opens the browser using selenium webdriver and navigates to the this week tab
    URL = "https://www.forexfactory.com/calendar?month=jan.2023"
#   URL = "https://www.forexfactory.com/calendar?month=feb.2011"
#   URL = "https://www.forexfactory.com/calendar?month=mar.2011"
#   URL = "https://www.forexfactory.com/calendar?month=apr.2011"
#   URL = "https://www.forexfactory.com/calendar?month=may.2011"
#   URL = "https://www.forexfactory.com/calendar?month=jun.2011"
#   URL = "https://www.forexfactory.com/calendar?month=jul.2011"
#   URL = "https://www.forexfactory.com/calendar?month=aug.2011"
#   URL = "https://www.forexfactory.com/calendar?month=sep.2011"
#   URL = "https://www.forexfactory.com/calendar?month=oct.2011"
#   URL = "https://www.forexfactory.com/calendar?month=nov.2011"
#   URL = "https://www.forexfactory.com/calendar?month=dec.2011"

    #open the browser
    browser = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

    #load page
    browser.get(URL)

    # Wait for the table element to load
    table = WebDriverWait(browser, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "calendar__table  "))
    )

    # Parse the HTML with Beautiful Soup
    soup = BeautifulSoup(browser.page_source, "html.parser") 
    
    
    # Extract the data from the table
    previous_date = ""
    d = {'Date': [], 'Time': [], 'Currency': [], 'Impact': [],'Description': [], 'Actual': [], 'Forecast': [], 'Previous': []}
    for row in soup.find_all("tr"):
        if "calendar__row" in row.get("class", []):
            date_cell = row.find("td", {"class": "calendar__date"})
            if date_cell is not None and date_cell.text.strip() != "":
                date = date_cell.text.strip()
                d['Date'].append(date)
                previous_date = date
            else:
                d['Date'].append(previous_date)
            time_cell = row.find("td", {"class": "calendar__time"})
            if time_cell is not None:
                d['Time'].append( time_cell.text.strip())
            else:
                d['Time'].append("")
            currency_cell = row.find("td", {"class": "calendar__currency"})
            if currency_cell is not None:
                d['Currency'].append(currency_cell.text.strip())
            else:
                d['Currency'].append( "")
            actual_cell = row.find("td", {"class": "calendar__actual"})
            if actual_cell is not None:
                d['Actual'].append(actual_cell.text.strip())
            else:
                d['Actual'].append("")
            forecast_cell = row.find("td", {"class": "calendar__forecast"})
            if forecast_cell is not None:
                d['Forecast'].append(forecast_cell.text.strip())
            else:
                d['Forecast'].append("")
            previous_cell = row.find("td", {"class": "calendar__previous"})
            if previous_cell is not None:
                d['Previous'].append(previous_cell.text.strip())
            else:
                d['Previous'].append("")
            event_cell = row.find("td", {"class": "calendar__event"})
            if event_cell is not None:
                d['Description'].append(event_cell.text.strip())
            else:
                d['Description'].append( "")
            impact_cell = row.find("td", {"class": "calendar__impact"})
            if impact_cell is not None:
                impact = impact_cell.find('span', {'title': True})
                if impact is not None:
                    d['Impact'].append(impact['title'])
                else:
                    d['Impact'].append("")
            else:
                d['Impact'].append("")
                


    forex_factory_df = pd.DataFrame.from_dict(d) 

    # Remove rows that only have date information
    forex_factory_df = forex_factory_df[forex_factory_df['Currency'] != '']




    filename = f'forex_factory.csv'
    #insert path of folder you want the csv to be in between r''
    forex_factory_df.to_csv(r'1' + '\\' + filename, header=True, index=False) 

    # Close the web driver
    browser.quit()

In [104]:
    scrape_forex_data()